[Jordà, Ò., & Taylor, A. M. (2016). The time for austerity: estimating the average treatment effect of fiscal policy. _The Economic Journal_, _126_(590), 219-255.](https://onlinelibrary.wiley.com/doi/abs/10.1111/ecoj.12332)

Replication Code Downloaded from https://sites.google.com/site/oscarjorda/home/local-projections

[Download this jupyternotebook from here](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/Replication.ipynb)

# Table of Contents
* [Initiating environment and downloading datasets](#initiating_env)
* [Preparation of dataset](#dataset)
* [Table 1](#table1)
* [Table 2 and A1](#tbl2A1)
     - [Table 2 - Panel (a)](#table2_a)
     - [Table 2 - Panel (b)](#table2_b)
     - [Table A1](#tableA1)
     - [Table A1 - Panel (a)](#tableA1_a)
     - [Table A1 - Panel (b)](#tableA1_b)
* [Table 3](#table3)
* [Table 4 and A2](#tbl4A2)
     - [Table 4](#table4)
     - [Table A2](#tableA2)
* [Table 5](#table5)
* [Table 6](#table6)

<a class="anchor" id="initiating_env"></a>

------
    
# Initiating environment and downloading datasets
---

In [1]:
#--------------------------------------------------------------------------------------------
# Necessary packages loading for python
#--------------------------------------------------------------------------------------------
import ipystata #package for executing stata code from python
import wget
from pathlib import Path
import os

In [2]:
#--------------------------------------------------------------------------------------------
# Downloading data from Github repository
#--------------------------------------------------------------------------------------------
path = 'https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_(2016)_The_time_for_austerity__REPLICATION_WORK/dataset/'
datasets = ['01_wdi_worldgdp.xlsx',
           '02_JST_panel2_v1.dta',
           '03_IMF-fiscalshocks.dta',
           '04_Leigh_database.dta',
           '05_IMFWEO_AUTIRL.dta',
           '06_03YR.xlsx',
           '07_aut_irl_8Apr2013.xlsx',
           '08_austria_gaggl.dta']

#creating data folder for downloading datasets
try:
    os.mkdir('data')
except:
    pass

for x in datasets:
    already_exists = len(list(Path().rglob(x))) #checking whether file exists or not
    
    if already_exists == 0:
        wget.download(path + x, 'data')

<a class='anchor' id='dataset'></a>

---
# Preparation of dataset (dataset.do)
---

In [3]:
%%capture
%%stata -os -cwd
clear
*#================================================================
*# importing excel data and saving it in tempfile for later use
*#================================================================
import excel  using data/01_wdi_worldgdp.xlsx, firstrow
tempfile wgdp
save `wgdp'

*#================================================================
* #read in the JST dataset
*#================================================================
use data/02_JST_panel2_v1.dta, clear
replace iso="PRT" if iso=="POR"

*#================================================================
* #read in the full IMF-GLP dataset from Leigh and merging it
*#================================================================
sort iso year
merge 1:1 iso year using data/03_IMF-fiscalshocks.dta
drop _merge //#dropping the column created on the merge process

*#================================================================
* #get the Leigh and AA variables
*#================================================================
sort iso year
tempfile data
save `data'

use data/04_Leigh_database.dta, clear
gen iso = wdicode
merge 1:1 iso year using `data'
drop _merge

*#================================================================
* #merge in AUT IRL data from WEO Oct 2012 to fill some gaps
* #to deal with missing data for AUT and IRL
*#================================================================
merge 1:1 iso year using data/05_IMFWEO_AUTIRL.dta
drop _merge
replace debtgdp = debtgdpweo/100 if iso=="IRL"|iso=="AUT"
gen rgdpnew = rgdpbarro
replace rgdpnew = rgdpweo if iso=="IRL"|iso=="AUT"

*#================================================================
* #cut down to the 17 country sample to match the IMF
*#================================================================
drop if iso=="CHE"
drop if iso=="NOR"

*#================================================================
* #read in world GDP back in here
*#================================================================
merge m:1 year using `wgdp'
drop _merge

In [4]:
%%capture
%%stata -o stata_df -os
*#================================================================
* #create treatment variable (fiscal consolidation action according to IMF)
*#================================================================
gen treatment = 0 if year>=1978 & year<=2007
replace treatment = 1 if total ~= . //# set treatment variable to 1 if total variable is not set to null
gen control = 1-treatment //#generating control variable which is opposite of treatment. Control means not received any treatment.

replace total = 0 if treatment == 0
replace tax = 0 if treatment == 0
replace spend = 0 if treatment == 0

In [5]:
%%stata -os
*#================================================================
* #defining panel dataset
*#================================================================
capture drop ccode //# does not show any error if drop ccode throws any error
egen ccode=group(iso)
sort iso year
xtset ccode year //#defining panel data


(1 missing value generated)

       panel variable:  ccode (unbalanced)
        time variable:  year, 1977 to 2011
                delta:  1 year



In [6]:
%%capture
%%stata -os
*#================================================================================
* #treatment happens at t=1 in IMF study setup, but known at t=0
* #use ftreatment as the indicator of policy choice at t=0 to match the IMF setup
*#=================================================================================
gen ftreatment = f.treatment

* #drop events around time of German reunification to match AA variable
replace ftreatment=. if iso=="DEU" & year ==1990 | iso=="DEU" & year ==1991


* #save to tempfile for later use
tempfile data
save `data'

In [7]:
%%capture
%%stata -os
*#================================================================================
* #more gaps to fill for broad datset
*** #read in short- and long-term interest rates for IRL // 
*** #short-term data from Israel Malkin
*** #long-term data from Alan
*#================================================================================
clear
import excel using data/06_03YR.xlsx, sheet(IRL) firstrow

gen d = date(date, "MDY")
format d %td
gen year = year(d)
rename IRL stir
collapse stir, by(year) //# creating average of stir variable by year
replace stir=16 if year == 1992 // #this is a fix for the code error in the data
replace stir = stir/100 
gen iso = "IRL"

tempfile sIRL
save `sIRL'
clear

import excel  using data/07_aut_irl_8Apr2013.xlsx, firstrow //# IRL ltrate from Alan
destring year, replace //#converting year from string to numerical value
keep year iso ltrate 
keep if iso=="IRL"
merge 1:1 iso year using `sIRL'
drop _merge
tempfile IRL
save `IRL'
clear

In [8]:
%%capture
%%stata -os
*#================================================================================
*** #read in data for AUT from Paul Gaggl
*#================================================================================
use data/08_austria_gaggl.dta
rename  at_i_3m_a stir
rename at_i_10y_a ltrate
replace stir = stir/100
replace ltrate = ltrate/100
merge 1:1 iso year using `IRL'
drop _merge

merge 1:1 iso year using `data'
drop _merge

gen lrgdp   = log(rgdpbarro) // #real GDP index from Barro
gen lrcon   = log(rconsbarro) // #real consumption index from Barro
gen lmoney  = log(money) // #M2, more or less
gen lstocks = log(stocks) // #Stock indices
gen lnarrow = log(narrowm) // #M1, more or less
gen cay = ca/gdp // #Current Account over GDP ratio
gen lloans  = log(loans1)

tempfile data
save `data'
clear

In [9]:
%%capture
%%stata -os
*#================================================================================
*** #read in data from Alan for AUT and IRL except for interest rates 
* #(goes in here because he already applied the necessary transformations)
*#================================================================================

import excel using data/07_aut_irl_8Apr2013.xlsx, firstrow
destring year, replace
drop ltrate
merge 1:1 year iso using `data'
drop _merge

replace ccode = 2 if ccode==. & iso=="AUT"
replace ccode = 11 if ccode==. & iso=="IRL"

replace cay = cay/100 if iso=="AUT" | iso=="IRL"
gen lcpi    = log(cpi)  //#CPI
gen lpop    = log(pop)

replace lrgdp = log(rgdp) - lpop if iso=="AUT" | iso=="IRL" //# log of per capita GDP

gen rprv  = lloans - lcpi - lpop  // #real per capita private loans
replace rprv = log(realloans) - lpop if iso=="AUT" | iso=="IRL"

gen riy = iy*rgdpbarro // #real per capita investment
replace riy = realinv/pop if iso=="AUT" | iso=="IRL"
gen lriy = log(riy)

gen rlmoney = lmoney - lcpi

sort ccode year
gen dlrgdp  = 100*d.lrgdp // #Annual real per capita GDP growth in percent
gen dlriy = 100*d.lriy // #Annual real per capita investment growth in percent
gen dlcpi   = 100*d.lcpi // #Annual inflation in percent
gen dlrcon  = 100*d.lrcon // #Annual real consumption growth in percent

gen drprv = 100*d.rprv // #Annual real per capita private loan growth 
gen drlmoney= 100*d.rlmoney  // #Annual Growth in M2 

replace cay = 100*cay
replace stir = 100*stir
replace ltrate = 100*ltrate 

In [10]:
%%capture
%%stata -os
*#================================================================================
* #match IMF: cumulate IMF real GDP growth rate (N=17) to recoup levels
* #g = growth of real GDP (OECD)
*#================================================================================
gen dlogy = log(1+g) if year>=1978 //# log(1+g) is to create log of Y difference variable i.e. growth of Y using log difference
by ccode: gen logyIMF=sum(dlogy) if year>=1978 // #creating cummulative column of dlogy
by ccode: replace logyIMF=0 if year==1977 // #start year =0


---
**datset now complete, now preparing for the empirical analysis**

---

In [11]:
%%capture
%%stata -os
*#================================================================================
* #define dependent variable and lags
*#================================================================================
gen ly = 100*logyIMF //# converted in % multiplying by 100. logyIMF is cummulative growth rate by ccode
gen dly = d.ly //#non-cummulative growth rate (I guess)
gen ldly = l.dly

*#================================================================================
* #generate lags of the broad set of controls
*#================================================================================
gen ldrprv = l.drprv 
gen ldlcpi = l.dlcpi
gen ldlriy = l.dlriy 
gen lstir = l.stir 
gen lltrate = l.ltrate 
gen lcay = l.cay

*#================================================================================
* #construct the HP filter of log y
*#================================================================================
drop if year > 2011

bysort ccode: hprescott ly, stub(HP) smooth(100) //#creates separate columns for each ccode
* #VERY HIGH smoothing (cf 6.25 Ravn-Uhlig)
* #but we want something like "output gap"

*#since the above code creates separate columns for HP extraction for each ccode,
*# the following code creates a single column for cyclical component
summarize ccode
local countries = r(max) //#number of countries

gen hply=.
forvalues i=1/`countries' {
	replace hply = HP_ly_`i' if ccode==`i'
}

*#================================================================================
* #now drop unwanted years after the HP filtering
* #all regressions are restricted to year>=1980 & year<=2007
*#================================================================================
drop if year<=1977
drop if year>=2008

*#================================================================================
* #bins - dividing whole dataset into group based on cyclical component of HP filter on Y
*#================================================================================
gen boom = cond(hply > +0,1,0) //#if positive boom
gen slump = 1 - boom

gen Hi = cond(hply > +1, 1, 0) //# if greater than +1 High, from +1 to -1 Medium and Less or equal to -1 low
gen Mid = cond(hply>-1 & hply<= +1, 1, 0)
gen Low = 1 - Hi - Mid

*#================================================================================
* #dummy interactions
*#================================================================================
gen ccodeLMH = ccode
replace ccodeLMH =  ccodeLMH+100 if Mid==1 //#greater than 100 ccode represents Medium, greater than 200 High, and less than 100 low
replace ccodeLMH =  ccodeLMH+200 if Hi==1
tabulate ccodeLMH, gen(ccodeLMHdum) //# generating dummy variable based on tabulation of ccodeLMH

*#================================================================================
* #dep vars for h-step ahead forecast (h=1,...,5)
*#================================================================================

local var ly ftreatment // #ftreatment depvar used in Table A5. storing ly and ftreatment variable in local macro named var

foreach v of local var { //#looping through var macro i.e. ly and ftreatment
    forvalues i=1/5 { //#looping through horizon 1 to 5

        if "`v'"=="ly" {
            gen `v'`i' = f`i'.`v' - `v' //#generating dependent variable, Y(t+h) - Y(t)
        }

        if "`v'"=="ftreatment" { //#creating ftreatment variable for horizon 
            gen `v'`i' = f`i'.`v'
        }
        label var `v'`i' "Year `i'" //#labelling newly created variable
    }
}

gen ly6 = (ly1+ly2+ly3+ly4+ly5)
label var ly6 "Sum" //# sum column for total effect

*#================================================================================
* # transform and interact AA dCAPB measure
*#================================================================================
gen AA = AA_dcapb * 100
gen fAA = f.AA
gen lfAA = l.fAA //# AA and lfAA is the same thing
gen fAAMid = fAA * Mid
gen fAALo = fAA * Lo

* #housekeeping
tab year, gen(y)
tabulate year, gen(dumyr)
tabulate iso, gen(dumiso)

*#================================================================================
* #dmdum = demeaned dummies
* #demeaned dummy variable
*#================================================================================
forvalues k = 1/17 {
    gen dmdumiso`k' = dumiso`k' - 1/17 //# mean is 1 divided by 17 (1/17). There is 17 countries.
}

*#================================================================================
* #dml0dly dml1dly = demeaned growth rates
* # demeaned growth rates
*#================================================================================
sum dly
gen dml0dly = dly - r(mean)

sum ldly
gen dml1dly = ldly - r(mean)


<a class='anchor' id='table1'></a>

---
# Table 1 (table1.do)
---

![Table1](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table1.PNG)

### For the summarized result of the code in the following cell, click the link below 

[Link to see summarized results](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table1_summarized.pdf)

In [12]:
%%stata -os
*#================================================================================
* #Table 1: Fiscal muliplier, d.CAPB, OLS estimate
*#================================================================================

gen lgfAA = 0
replace lgfAA = fAA if abs(fAA)>1.5 //# Fiscal multiplier, large change in CAPB (> 1.5%)

gen smfAA = 0
replace smfAA = fAA if abs(fAA)<=1.5 //# Fiscal multiplier, small change in CAPB (<= 1.5%)

*** #OLS AA all changes in dCAPB full sample
label var fAA "Fisc multiplier. Full sample"
label var lgfAA "Fisc multiplier. Large Cons."
label var smfAA "Fisc multiplier. Small Cons."


forvalues i=1/6 {
    * #the dummy for the U.S. is dropped to avoid collinearity with the constant
    * #specification a la AA
    reg ly`i'   fAA ///
        hply dml0dly dml1dly dmdumiso1-dmdumiso16 ///
        if year>=1980 & year<=2007,  cluster(iso) 
   
    reg ly`i'   smfAA lgfAA  ///
        hply dml0dly dml1dly dmdumiso1-dmdumiso16  ///
        if year>=1980 & year<=2007,  cluster(iso) 

}


(159 real changes made, 19 to missing)

(351 real changes made)

Linear regression                               Number of obs     =        457
                                                F(3, 16)          =          .
                                                Prob > F          =          .
                                                R-squared         =     0.6087
                                                Root MSE          =      1.218

                                   (Std. Err. adjusted for 17 clusters in iso)
------------------------------------------------------------------------------
             |               Robust
         ly1 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         fAA |   .1148758   .0403991     2.84   0.012     .0292337     .200518
        hply |   -.493521    .038125   -12.94   0.000    -.5743423   -.4126996
     dml0dly |   .5772986   

<a class='anchor' id='tbl2A1'></a>

---
# Table 2 and A1 (table2andA1.do)
---

![Table2](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table2.PNG)

<a class='anchor' id='table2_a'></a>

## Table 2 - Panel (a)

[Click here for summarized result of code below](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table2-panel%28a%29.pdf)

In [13]:
%%stata -os
* #================================================================================
* #Table 2: Fiscal muliplier, d.CAPB, OLS estimate boom/slump
* #================================================================================

* # replicating Table 2 - Panel (a) Uniform effect of d.CAPB changes
forvalues i=1/6 {
    foreach c in boom slump {
        * #the dummy for the U.S. is dropped to avoid collinearity with the constant
        reg ly`i'   fAA ///
            hply dml0dly dml1dly dmdumiso1-dmdumiso16 ///
            if `c' == 1 & year >= 1980 & year <= 2007, cluster(iso)
    }
}

. forvalues i=1/6 {
  2.     foreach c in boom slump {
  3.         * #the dummy for the U.S. is dropped to avoid collinearity with the constant
  4.     }
  5. }

Linear regression                               Number of obs     =        222
                                                F(3, 16)          =          .
                                                Prob > F          =          .
                                                R-squared         =     0.6310
                                                Root MSE          =     1.2427

                                   (Std. Err. adjusted for 17 clusters in iso)
------------------------------------------------------------------------------
             |               Robust
         ly1 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         fAA |   .2109091    .067535     3.12   0.007     .0677414    .3540768
        h

<a class='anchor' id='table2_b'></a>

## Table 2 - Panel (b)

[Click here for summarized result of code below](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table2-panel%28b%29.pdf)

In [14]:
%%stata -os
* # replicating Table 2 - Panel (b) Separate effects of d.CAPB for Large( >1.5%) and Small (<= 1.5%) changes
forvalues i=1/6 {
    foreach c in boom slump {
        reg ly`i'   smfAA lgfAA ///
            hply dml0dly dml1dly dmdumiso1-dmdumiso16 ///
            if `c' == 1 & year >= 1980 & year <= 2007, cluster(iso)
    }
}

. forvalues i=1/6 {
  2.     foreach c in boom slump {
  3.         reg ly`i'   smfAA lgfAA hply dml0dly dml1dly dmdumiso1-dmdumiso16 if `c' == 1 & year >= 1980 & yea
  4.     }
  5. }

Linear regression                               Number of obs     =        222
                                                F(4, 16)          =          .
                                                Prob > F          =          .
                                                R-squared         =     0.6330
                                                Root MSE          =     1.2424

                                   (Std. Err. adjusted for 17 clusters in iso)
------------------------------------------------------------------------------
             |               Robust
         ly1 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       smfAA |      .0637   .1116935     0.57   0.576    -.1730796

<a class='anchor' id='tableA1'></a>

![TableA1](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/tableA1.PNG)

<a class='anchor' id='tableA1_a'></a>

## Table A1 - Panel (a)

[Click here for summarized result of code below](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/tableA1-panel%28a%29.pdf)

In [15]:
%%stata -os
* #================================================================================
* #Table A1: Fiscal muliplier, d.CAPB, OLS estimate boom/slump (world gdp included)
* #================================================================================

* # replicating Table 2 - Panel (a) Uniform effect of d.CAPB changes
forvalues i=1/6 {
    foreach c in boom slump {
        * #the dummy for the U.S. is dropped to avoid collinearity with the constant
        reg ly`i'   fAA ///
            hply dml0dly dml1dly dmdumiso1-dmdumiso16 wgdp ///
            if `c' == 1 & year >= 1980 & year <= 2007, cluster(iso)
    }
}

. forvalues i=1/6 {
  2.     foreach c in boom slump {
  3.         * #the dummy for the U.S. is dropped to avoid collinearity with the constant
  4.     }
  5. }

Linear regression                               Number of obs     =        222
                                                F(4, 16)          =          .
                                                Prob > F          =          .
                                                R-squared         =     0.6684
                                                Root MSE          =      1.181

                                   (Std. Err. adjusted for 17 clusters in iso)
------------------------------------------------------------------------------
             |               Robust
         ly1 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         fAA |   .2103142   .0677412     3.10   0.007     .0667092    .3539191
        h

<a class='anchor' id='tableA1_b'></a>

## Table A1 - Panel (b)

[Click here for summarized result of code below](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/tableA1-panel%28b%29.pdf)

In [16]:
%%stata -os
* # replicating Table 2 - Panel (b) Separate effects of d.CAPB for Large( >1.5%) and Small (<= 1.5%) changes
forvalues i=1/6 {
    foreach c in boom slump {
        reg ly`i'   smfAA lgfAA ///
            hply dml0dly dml1dly dmdumiso1-dmdumiso16 wgdp ///
            if `c' == 1 & year >= 1980 & year <= 2007, cluster(iso)
    }
}

. forvalues i=1/6 {
  2.     foreach c in boom slump {
  3.         reg ly`i'   smfAA lgfAA hply dml0dly dml1dly dmdumiso1-dmdumiso16 wgdp if `c' == 1 & year >= 1980 
  4.     }
  5. }

Linear regression                               Number of obs     =        222
                                                F(5, 16)          =          .
                                                Prob > F          =          .
                                                R-squared         =     0.6709
                                                Root MSE          =     1.1795

                                   (Std. Err. adjusted for 17 clusters in iso)
------------------------------------------------------------------------------
             |               Robust
         ly1 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       smfAA |    .044735   .1170588     0.38   0.707    -.2034186

<a class='anchor' id='table3'></a>

---
# Table 3 (table3.do)
---

![Table3](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table3.PNG)

[Click here for summarized result of code below](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table3.pdf)

<span style="color:red">Standard Errors (SEs) for coefficients are slightly different than what have been published in the paper. [The included code's compiled tables also shows slightly different SEs.](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/tables_figures_compiled.pdf) However, the key result does not change. </span>

![](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table3_included.PNG)

In [17]:
%%stata -os
* #================================================================================
* # Table 3: Fiscal multiplier, d.CAPB, IV estimates. Log real GDP (relative to Year 0, x 100)
* #================================================================================

forvalues i = 1/6   {
    foreach v in treatment total {
        * the dummy for the U.S. is dropped to avoid collinearity with the constant
        ivreg2 ly`i'   (fAA= f.`v') ///
            hply dml0dly dml1dly dmdumiso1-dmdumiso16 ///
            if year>=1980 & year<=2007, cluster(iso)

    }
}

. forvalues i = 1/6   {
  2.     foreach v in treatment total {
  3.         * the dummy for the U.S. is dropped to avoid collinearity with the constant
  4. 
.     }
  5. }

IV (2SLS) estimation
--------------------

Estimates efficient for homoskedasticity only
Statistics robust to heteroskedasticity and clustering on iso

Number of clusters (iso) =          17                Number of obs =      457
                                                      F( 20,    16) =    19.69
                                                      Prob > F      =   0.0000
Total (centered) SS     =  1652.910839                Centered R2   =   0.4727
Total (uncentered) SS   =  4677.500367                Uncentered R2 =   0.8137
Residual SS             =  871.6510062                Root MSE      =    1.381

------------------------------------------------------------------------------
             |               Robust
         ly1 |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
----

<a class='anchor' id='tbl4A2'></a>

---
# Table 4 and A2 (table4andA2.do)
---

<a class='anchor' id='table4'></a>

## Table 4

![](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table4.PNG)

[Click here for summarized result of code below](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table4.pdf)

<span style="color:red">Standard Errors (SEs) for coefficients are slightly different than what have been published in the paper. [The included code's compiled tables also shows slightly different SEs.](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/tables_figures_compiled.pdf) However, the key result does not change. </span>

![](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table4_included.PNG)

In [18]:
%%stata -os
* #================================================================================
* #Table 4: Fiscal multiplier, d.CAPB, IV estimate (binary), boom/slump
* #================================================================================

foreach c in boom slump {
    gen z`c'=f.treatment*`c'
}

forvalues i = 1/6   {
    foreach c in boom slump {
        * #the dummy for the U.S. is dropped to avoid collinearity with the constant
        ivreg2 ly`i'   (fAA= zboom zslump) ///
            hply dml0dly dml1dly dmdumiso1-dmdumiso16 ///
            if `c'==1 & year>=1980 & year<=2007,  cluster(iso) 
    }
}

(17 missing values generated)
(17 missing values generated)

. forvalues i = 1/6   {
  2.     foreach c in boom slump {
  3.         * #the dummy for the U.S. is dropped to avoid collinearity with the constant
  4.     }
  5. }
Warning - collinearities detected
Vars dropped:       zslump

IV (2SLS) estimation
--------------------

Estimates efficient for homoskedasticity only
Statistics robust to heteroskedasticity and clustering on iso

Number of clusters (iso) =          17                Number of obs =      222
                                                      F( 20,    16) =     6.49
                                                      Prob > F      =   0.0002
Total (centered) SS     =  841.2086444                Centered R2   =   0.4100
Total (uncentered) SS   =  1897.843827                Uncentered R2 =   0.7385
Residual SS             =  496.3132301                Root MSE      =    1.495

------------------------------------------------------------------------------
    

<a class='anchor' id='tableA2'></a>

## Table A2

![](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/tableA2.PNG)

[Click here for summarized result of code below](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/tableA2.pdf)

<span style="color:red">Standard Errors (SEs) for coefficients are slightly different than what have been published in the paper. [The included code's compiled tables also shows slightly different SEs.](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/tables_figures_compiled.pdf) However, the key result does not change. </span>

![](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/tableA2_included.PNG)

In [19]:
%%stata -os

forvalues i = 1/6   {
    foreach c in boom slump {
        * #the dummy for the U.S. is dropped to avoid collinearity with the constant
        ivreg2 ly`i'   (fAA= zboom zslump) ///
            hply wgdp dml0dly dml1dly dmdumiso1-dmdumiso16 ///
            if `c'==1 & year>=1980 & year<=2007,  cluster(iso) 
    }
}


. forvalues i = 1/6   {
  2.     foreach c in boom slump {
  3.         * #the dummy for the U.S. is dropped to avoid collinearity with the constant
  4.     }
  5. }
Warning - collinearities detected
Vars dropped:       zslump

IV (2SLS) estimation
--------------------

Estimates efficient for homoskedasticity only
Statistics robust to heteroskedasticity and clustering on iso

Number of clusters (iso) =          17                Number of obs =      222
                                                      F( 21,    16) =    14.11
                                                      Prob > F      =   0.0000
Total (centered) SS     =  841.2086444                Centered R2   =   0.4642
Total (uncentered) SS   =  1897.843827                Uncentered R2 =   0.7625
Residual SS             =  450.7263607                Root MSE      =    1.425

------------------------------------------------------------------------------
             |               Robust
         ly1 |      Coef.   

<a class='anchor' id='table5'></a>

---
# Table 5 (table5.do)
---

![](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table5.PNG)

[Click here for summarized result of code below](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table5.pdf)

<span style="color:red"> Coefficients are significant at 1% level of significance but only has one star ( * ) instead of three stars ( *** ). Could be a typo. [The included code's compiled tables also shows significant at 1% level of significance.](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/tables_figures_compiled.pdf) </span>

![](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table5_included.PNG)

In [21]:
%%stata -os
* #================================================================================
* #Table 5 / Balance check
* #================================================================================

* # This conducts ttest for checking whether control and treatment groups have statistically different means
* # Doing it one by one.

gen fcontrol = 1-ftreatment // #ttest will use control as the reference group

foreach xx in debtgdp hply dly treatment {
	ttest `xx', by(fcontrol)
}

* # OR, you can use the combined one-line code provided by the author to run for each variables combinedly.
eststo clear
estpost ttest debtgdp hply dly treatment, by(fcontrol)



(19 missing values generated)

Two-sample t test with equal variances
------------------------------------------------------------------------------
   Group |     Obs        Mean    Std. Err.   Std. Dev.   [95% Conf. Interval]
---------+--------------------------------------------------------------------
       0 |     168    .6863568    .0247139    .3203284    .6375649    .7351487
       1 |     319    .5573135    .0146543    .2617334    .5284819     .586145
---------+--------------------------------------------------------------------
combined |     487    .6018295    .0131219    .2895747    .5760468    .6276121
---------+--------------------------------------------------------------------
    diff |            .1290433    .0270042                .0759836     .182103
------------------------------------------------------------------------------
    diff = mean(0) - mean(1)                                      t =   4.7786
Ho: diff = 0                                     degrees of 

<a class='anchor' id='table6'></a>

---
# Table 6 (table6.do)
---

![](https://github.com/htdanil/referenced_to_github/raw/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table6.PNG)

[Click here for summarized result of code below](https://github.com/htdanil/referenced_to_github/blob/master/GF0004_Jorda_Taylor_%282016%29_The_time_for_austerity__REPLICATION_WORK/results/table6.pdf)

In [23]:
%%stata -os

* #================================================================================
* #Table 6: Omitted Variables Explain Output Fluctuations
* #================================================================================

local instrument treatment total //# instrument variables

local var_list dly drprv dlcpi dlriy stir ltrate cay //# omitted variables to be tested

* # creating variables for result storage
gen var="."
gen ols=.
gen iv_treatment=.
gen iv_total=.

* # storing variables name in var column(variable)
local count 1
foreach v of local var_list {
	replace var="`v'" if _n==`count'
	local count = `count' + 1
}	


* # running OLS
foreach v of local var_list {

xtreg ly1 hply fAA dly ldly `v' l`v'  if year>=1980 & year<=2007, fe vce(cluster iso)
	test (`v'=0) (l`v'=0)
	
	* # results are stored in scalars and we can see the list of scalars by using command 
	* # return list
	replace ols=round(r(p), 0.01) if var=="`v'" //# p-value for model test can be obtained using r(p)
	
}

* # running IV model for both treatment(binary) and total(continuous)
foreach z of local instrument {
	foreach v of local var_list {
		
		xtivreg2 ly1 hply (fAA=f.`z') `v' l`v'  if year>=1980 & year<=2007, fe cluster(iso)
			test (`v'=0) (l`v'=0)

		* # results are stored in scalars and we can see the list of scalars by using command 
		* # return list
		replace iv_`z'=round(r(p), 0.01) if var=="`v'" //# p-value for model test can be obtained using r(p)
	}
}

* # listing the stored results in tabular form
list var ols iv_treatment iv_total if _n < `count'


(510 missing values generated)

(510 missing values generated)

(510 missing values generated)

variable var was str1 now str3
(1 real change made)
variable var was str3 now str5
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
variable var was str5 now str6
(1 real change made)
(1 real change made)

note: dly omitted because of collinearity
note: ldly omitted because of collinearity

Fixed-effects (within) regression               Number of obs      =       457
Group variable: ccode                           Number of groups   =        17

R-sq:  within  = 0.5393                         Obs per group: min =        25
       between = 0.9939                                        avg =      26.9
       overall = 0.5951                                        max =        27

                                                F(4,16)            =    451.02
corr(u_i, Xb)  = 0.3547                         Prob > F           =    0.0000

                    